In [25]:
import pandas as pd 
import numpy as np
import pickle
from src.utils import parse_date_DOB,parse_date_for_tenure ,calculate_age , calculate_tenure


In [26]:
with open("model.pkl" , "rb") as file_obj:
   model= pickle.load(file_obj )

In [27]:
model

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [28]:
print("Model expects input shape:", model.n_features_in_)


Model expects input shape: 305


In [29]:
test_df =pd.read_csv(r"artifact\\11032024__081113\\data_ingestion\\dataset\\test.csv")
#selecting impotant feature for model training
important_features=["Employee_Name" , 
                    "GenderID" ,"Salary" ,
                    "Termd" ,"Position" ,
                    "State" ,"DOB" ,"DateofHire" ,
                    "DateofTermination" ,
                    "PerformanceScore" ,
                    "EngagementSurvey" ,
                    "EmpSatisfaction" ,
                    "Absences" , "ManagerName", "Zip" ,
                    "SpecialProjectsCount" ,
                    "HispanicLatino","Department" ,
                    'MarriedID']
not_important_features = [ column for column in test_df.columns if column not in important_features]

In [30]:
test_df.drop(not_important_features, axis=1 , inplace=True) #Lets drop unnecessary column
test_df["DOB"] = test_df["DOB"].apply(parse_date_DOB)
test_df['Age'] = test_df['DOB'].apply(calculate_age)
mean_age = test_df['Age'][test_df['Age'] <= 50].mean()
test_df['Age'] = test_df['Age'].apply(lambda x: mean_age if x > 50 else x)

test_df['DateofHire'] = test_df['DateofHire'].apply(parse_date_for_tenure)
test_df['DateofTermination'] = test_df['DateofTermination'].apply(parse_date_for_tenure)
test_df['Tenure'] = test_df.apply(calculate_tenure, axis=1)
test_df.drop(["DOB" , "DateofHire" ,"DateofTermination"] ,axis=1 ,inplace=True)

In [31]:
with open("transformation.pkl" ,"rb") as file_obj:
    transformer=pickle.load(file_obj)

In [ ]:
x=test_df.drop("Termd" , axis=1)
y=test_df["Termd"]
nominal_features = ['Employee_Name', 'Position', 'Department', 'ManagerName', 'PerformanceScore' ,"State" ,"HispanicLatino"]
x